In [75]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
from tqdm import tqdm, tqdm_notebook
from pathlib import Path
from collections import Counter
sns.set()

os.chdir('../..')

DATA = Path('data')
RAW  = DATA/'raw'
PROCESSED = DATA/'processed'
SUBMISSIONS = DATA/'submissions'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
%%time
product           = pd.read_csv(RAW/'productid_category.csv', low_memory=False)
train_tracking    = pd.read_csv(RAW/'train_tracking.csv', low_memory=False)
test_tracking     = pd.read_csv(RAW/'test_tracking.csv', low_memory=False)
train_session     = pd.read_csv(RAW/'train_session.csv', low_memory=False)
test_session      = pd.read_csv(RAW/'random_submission.csv', low_memory=False)

CPU times: user 1min 7s, sys: 7.2 s, total: 1min 14s
Wall time: 1min 14s


In [37]:
category.shape, train_tracking.shape, test_tracking.shape, train_session.shape, test_session.shape

((10635919, 4), (1355095, 31), (908233, 31), (133123, 2), (88750, 2))

In [133]:
train_session.head()

,sid,target
0,U6clt0UYaQB7vJQFmSWYymsAfwmT9SMhrm2oXQ8TC5M8mf...,False
1,0qRbLkxeOjeEo6CtxtB6VI8FNPt4Vl8niySzZLqSWiQOka...,False
2,cnCRGomfPqEWuhMZkA0RQfvTQiwVoyHXUqvqnPCutVouFO...,False
3,zBwrofN0r2ps9u/UCUS134SiZIqB+UgIEr0MZGHzksqR4f...,False
4,Kbr60r9eaX/ENOJ81R1YBxfwrEohoxyQI9Ma0fODsS+/XN...,False


# Changing duration to seconds [\\\\overwrite//]

In [38]:
train_tracking.duration = pd.to_timedelta(train_tracking.duration).dt.total_seconds()

# Extracting page location [tracking.page]

In [56]:
def extract_page(x):
    pages_types = ['_LR', '_PA', '_LP', '_CAROUSEL', '_SHOW_CASE']
    pages = ['CAROUSEL', 'PA', 'SEARCH', 'SHOW_CASE', 'LIST_PRODUCT']
    pages_map = [['PURCHASE_PRODUCT_UNKNOW_ORIGIN', 'UNKNOWN']]
    for pages_type in pages_types:
        if x.endswith(pages_type):
            return x[-len(pages_type)+1:]
    for page in pages:
        if x == page:
            return x
    for page_map in pages_map:
        if x == page_map[0]:
            return page_map[1]
    return '::' + x
test_tracking['page'] = test_tracking.type.apply(extract_page)

In [60]:
set(test_tracking.page.values)

{'CAROUSEL',
 'LIST_PRODUCT',
 'LP',
 'LR',
 'PA',
 'SEARCH',
 'SHOW_CASE',
 'UNKNOWN'}

# Simplifying categories [product.cat1]

In [110]:
counter1 = product.groupby('category_product_id_level1').size()
counter1dict = counter1.to_dict()

In [112]:
value_count = counter1.value_counts()

In [128]:
OTHERS = 10e7
mapcat = {}
for idx in counter1dict:
    if counter1dict[idx] > 10:
        mapcat[idx] = idx
    else:
        mapcat[idx] = OTHERS

In [129]:
len(set(mapcat.values()))

61

In [130]:
product['cat1'] = product.category_product_id_level1.apply(lambda x: mapcat[x])

In [131]:
product.cat1.describe()

count    1.063592e+07
mean     4.798615e+07
std      4.995942e+07
min      0.000000e+00
25%      2.100000e+01
50%      4.400000e+01
75%      1.000000e+08
max      1.000000e+08
Name: cat1, dtype: float64

# Adding category feature to tracking